In [1]:
from pandas import read_csv
df = read_csv("npw_products.csv", encoding="utf-8")
df


,URL,type,description
0,https://napieknewlosy.pl/products/hairmoji-chi...,"<div class=""pl-group pl-flex-column group_top_...",HAIRMOJI Chill regenerująca odżywka 200 ml\nPo...
1,https://napieknewlosy.pl/products/hairmoji-tea...,"<div class=""pl-group pl-flex-column group_top_...",HAIRMOJI Tea Spill codzienna odżywka 200 ml\nP...
2,https://napieknewlosy.pl/products/barwa-nablys...,"<div class=""pl-group pl-flex-column group_top_...","BARWA NATURALNA opracowała specjalną odżywkę, ..."
3,https://napieknewlosy.pl/products/hairmoji-swo...,"<div class=""pl-group pl-flex-column group_top_...",HAIRMOJI Swoosh Lekka odżywka 200 ml\nPotrzebu...
4,https://napieknewlosy.pl/products/gosh-odzywka...,"<div class=""pl-group pl-flex-column group_top_...",GOSH Odżywka do włosów Pump up the Volume zost...
...,...,...,...
176,https://napieknewlosy.pl/products/olaplex-no-5...,"<div class=""pl-group pl-flex-column group_top_...",OLAPLEX No.5 Bond Maintenance Conditioner to i...
177,https://napieknewlosy.pl/products/anwen-emolie...,"<div class=""pl-group pl-flex-column group_top_...",Emolientowa Odżywka do włosów o niskiej porowa...
178,https://napieknewlosy.pl/products/garnier-fruc...,"<div data-label-id=""52659"" class=""pl-animation...",GARNIER FRUCTIS Hair Food Banana Maska do włos...
179,https://napieknewlosy.pl/products/health-labs-...,"<div data-label-id=""52659"" class=""pl-animation...",HEALTH LABS Odżywka kondycjonująca do łodygi w...


In [ ]:
import regex
from typing import Optional

def extract_type(text: str) -> Optional[str|None]:
    """
    Gets type of the conditioner.

    Args:
        text: data from the 'type' column

    Returns:
        conditioner_type
    """
    regex_match = regex.match(r'^.*src="\s*([^"]*?)\s*".*?$', text)
    if regex_match:
        if regex.findall(r'\b6673f6f5265e4\b', regex_match.group(1)):
            conditioner_type = "EH"
        elif regex.findall(r'\b6673f70290fe3\b', regex_match.group(1)):
            conditioner_type = "PE"
        elif regex.findall(r'\b6673f71587af5\b', regex_match.group(1)):
            conditioner_type = "PEH"
        elif regex.findall(r'\b6673f76bb7843\b', regex_match.group(1)):
            conditioner_type = "PH"
        elif regex.findall(r'\b6673f77ad743c\b', regex_match.group(1)):
            conditioner_type = "E"
        elif regex.findall(r'\b6673f787a4726\b', regex_match.group(1)):
            conditioner_type = "H"
        elif regex.findall(r'\b6673f7964a532\b', regex_match.group(1)):
            conditioner_type = "P"
        else:
            conditioner_type = None
    else:
        conditioner_type = None   
    return conditioner_type

def extract_ingredients(text: str) -> Optional[str|None]:
    """
    Gets ingredients of a conditioner from the product description.
    
    Args:
        text: data from the 'description' column

    Returns:
        ingredients
    """
    regex_match = regex.search(r'\b(?:Sk[łl]ad|INCI|Sk[łl]adniki)\b\s*:\s*(.*)', text, regex.DOTALL)
    if regex_match:
        description: str = regex.sub(r'(?i)(\n\s*)*\b(pojemno[śs][ćc]|UNDER YOUR? SKIN|Produkt|Niezgodn|zgodn|Sk[łl]ad).*', '', regex_match.group(1), regex.DOTALL)
        description = regex.sub(r'\((?!Iso)[^\)]*?\)|\[[^\]]*?\]', '', description)
        description = regex.sub(r'&|\.', ', ', description)
        description = regex.sub(r'[^-\w\s/,\(\)\.]+', '', description)
        description = regex.sub(r'(\d+)\s*,\s*(\d+)', r'\1.\2', description)
        description = regex.sub(r'(?i)\bNo\b\s*,', 'No.', description)
        description = regex.sub(r'\s+', ' ', description)
        ingredients: str = regex.sub(r'(?:\s*,\s*){2,}|\A(?:\s*,\s*)+|(?:\s*,\s*)+\Z', '', description)
        if ingredients != "":
            return ingredients
        else:
            return None
    else:
        return None

# Applying the function on the column 'type'
df['type'] = df['type'].apply(extract_type)
df['description'] = df['description'].apply(extract_ingredients)
df.dropna(how='any', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 0 to 180
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   URL          179 non-null    object
 1   type         179 non-null    object
 2   description  179 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [3]:
df.to_csv("products_cleaned.csv", index=False, encoding="utf-8")